## 88 Project: Narrative Pace in "A Passage to India"

## Code Preparation

In [ ]:
import numpy as np

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
import string
from string import punctuation 

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from urllib.request import urlopen 
from collections import Counter
from datascience import *
import re

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

## Data Preparation

In [ ]:
with open('APassageToIndiaText.txt', 'r') as file_in: 
    passage_string = file_in.read()
#With open opens and closes the file 
#File_in = varible

In [ ]:
#passage_string.replace('\n', ' ') 
#Replaces '|n'/line break with a space
#Can also replace double space with one space

In [ ]:
passage_string = passage_string.lower()
passage_tokens = word_tokenize(passage_string)

In [ ]:
#passage_new = "".join([char for char in passage_tokens if char not in string.punctuation]).split()
#passage_new

In [ ]:
from collections import Counter
passage_dict = Counter(passage_tokens)

In [ ]:
passage_dict.keys()

In [ ]:
passage_dict.values()

In [ ]:
passage_dict.most_common(2)

In [ ]:
len(passage_tokens) / len(passage_dict.keys())
#Lexical diversity of the novel! 

## Sentiment Words

In [ ]:
with open('PositiveWordstext.txt') as file_in: 
    positive_sentiment_words = file_in.read()
positive_words = positive_sentiment_words.split()
positive_dict = Counter(positive_words)
positive_keys = positive_dict.keys()
positive_stems = [stemmer.stem(token) for token in positive_keys]

In [ ]:
with open('NegativeWordstext.txt') as file_in: 
    negative_sentiment_words = file_in.read()
negative_words = negative_sentiment_words.split()
negative_dict = Counter(negative_words)
negative_keys = negative_dict.keys()
negative_stems = [stemmer.stem(token) for token in negative_keys]

In [ ]:
#Need to break up text, compare each segment to list of positive or negative words, then count how many positive or negative words
#are in the list, then map that across the text

# Counting Sentiment Words by Chapter

In [ ]:
chapters_list = passage_string.split('chapter')
del chapters_list[0]

In [ ]:
chapters_array = np.array(chapters_list)
len(chapters_array)

In [ ]:
chapter_label = ['Chapter ' + str(i+1) for i in range(len(chapters_array))]
chapter_text = [i for i in chapters_list]
chapter_table = Table().with_column('Chapter', chapter_label).with_column('Segment', chapter_text)
chapter_table

In [ ]:
def negative(string): 
    word_list = []
    for i in string.split(): 
        if stemmer.stem(i) in negative_stems: 
            word_list.append(i)
    return word_list

def positive(string): 
    word_list = []
    for i in string.split():
        if stemmer.stem(i) in positive_stems:
            word_list.append(i)
    return word_list

In [ ]:
negative_chapter_words = chapter_table.apply(negative, 'Segment')
positive_chapter_words = chapter_table.apply(positive, 'Segment')
negative_positive_chapter_table = chapter_table.with_column('Negative Words', negative_chapter_words).with_column(
                                                            'Positive Words', positive_chapter_words)
negative_chapter_words_len = [len(i) for i in negative_positive_chapter_table.column('Negative Words')]
positive_chapter_words_len = [len(i) for i in negative_positive_chapter_table.column('Positive Words')]
negative_positive_chapter_total = np.array(negative_chapter_words_len) + np.array(positive_chapter_words_len)
almost_final_chapter_table = negative_positive_chapter_table.with_column(
                            'Negative Count', negative_chapter_words_len).with_column(
                            'Positive Count', positive_chapter_words_len).with_column(
                            'Total Count', negative_positive_chapter_total)
chapter_length = [len(i) for i in almost_final_chapter_table.column('Segment')]
chapter_percentage = negative_positive_chapter_total / chapter_length
final_chapter_table = almost_final_chapter_table.with_column('Percentage', chapter_percentage)
final_chapter_table.show(5)

## Counting Sentiment Words by Sentence

In [ ]:
#Example of old code: 
#sentence_list = passage_string.split('.')
#len(sentence_list)
#sentence_array = np.array(sentence_list)
#sentence_array

In [ ]:
sentence_list = sent_tokenize(passage_string)

In [ ]:
sentence_number = [(i+1) for i in range(len(sentence_list))]
sentence_text = [i for i in sentence_list]
sentence_table = Table().with_column('Sentence Number', sentence_number).with_column('Sentence Text', sentence_text)
sentence_table

In [ ]:
def negative(string): 
    word_list = []
    for i in string.split(): 
        if stemmer.stem(i) in negative_stems: 
            word_list.append(i)
    return word_list
negative('an ugly dirty ugly bad dog')

In [ ]:
def positive(string): 
    word_list = []
    for i in string.split():
        if stemmer.stem(i) in positive_stems:
            word_list.append(i)
    return word_list
positive('a beautiful happy happy wonderful dog')

In [ ]:
negative_sentence_words = sentence_table.apply(negative, 'Sentence Text')
positive_sentence_words = sentence_table.apply(positive, 'Sentence Text')
almost_final_sentence_table = sentence_table.with_column('Negative Words', negative_sentence_words).with_column(
                                                'Positive Words', positive_sentence_words)
almost_final_sentence_table
negative_sentence_words_len = [len(i) for i in almost_final_sentence_table.column('Negative Words')]
positive_sentence_words_len = [len(i) for i in almost_final_sentence_table.column('Positive Words')]
negative_positive_sentence_total = np.array(positive_sentence_words_len) + np.array(negative_sentence_words_len)
sentence_length = [len(i.split()) for i in almost_final_sentence_table.column('Sentence Text')]
sentence_percentage = negative_positive_sentence_total / sentence_length
final_sentence_table = almost_final_sentence_table.with_column('Negative Count', negative_sentence_words_len).with_column(
                                                                'Positive Count', positive_sentence_words_len).with_column(
                                                                'Total', negative_positive_sentence_total).with_column(
                                                                'Percentage', sentence_percentage)
final_sentence_table.show(5)

In [ ]:
#At this point I have produced tables that tally up the negative and positive words in each chapter in the text and each sentence
#In the text.
#Next Steps: 
#Look at concrete words. 
#Put everything in standard units 
#Plot the data
#Study the maps, compare the maps, see what inferences can be drawn
#Look at outliers, those areas that deviate pretty strongly from the average 

## Counting Concrete Words by Sentence and Chapter

In [ ]:
words = Table.read_table('Data8ConcreteCSV.csv')
concrete_table = words.where(words.column('Object') == 'Object')
concrete_array = concrete_table.column('Entry')
concrete_array
concrete_string = str(concrete_array)
concrete_lower = concrete_string.lower()
concrete_tokens = word_tokenize(concrete_lower)
concrete_tokens = ["".join([char for char in word if char.isalpha()]) for word in concrete_tokens]
concrete_stems = [stemmer.stem(token) for token in concrete_tokens]

In [ ]:
def concrete(string): 
    word_list = []
    for i in string.split():
        if stemmer.stem(i) in concrete_stems:
            word_list.append(i)
    return word_list

concrete('a big bag of ammunition')

In [ ]:
concrete_words_sentence = sentence_table.apply(concrete, 'Sentence Text')
concrete_words_sentence_table = sentence_table.with_column('Concrete Words', concrete_words_sentence)
concrete_lengths_sentence = [len(i) for i in concrete_words_sentence_table.column('Concrete Words')]
concrete_percentage_sentence = np.array(concrete_lengths_sentence) / sentence_length
concreteness_table_sentence = concrete_words_sentence_table.with_column(
                            'Total Count', concrete_lengths_sentence).with_column(
                            'Percentage', concrete_percentage_sentence)
concreteness_table_sentence.show(5)

In [ ]:
concrete_words_chapter = chapter_table.apply(concrete, 'Segment')
concrete_words_chapter_table = chapter_table.with_column('Concrete Words', concrete_words_chapter)
concrete_lengths_chapter = [len(i) for i in concrete_words_chapter_table.column('Concrete Words')]
concrete_percentage_chapter = np.array(concrete_lengths_chapter) / np.array(chapter_length)
concreteness_table_chapter = concrete_words_chapter_table.with_column('Total Count', concrete_lengths_chapter).with_column(
                                                                        'Percentage', concrete_percentage_chapter)
concreteness_table_chapter.show(5)

## Standard Units for Sentiment Words

In [ ]:
def standard_units(any_numbers):
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)

In [ ]:
print('Average Percentage of Sentiment Words:') 
print(np.mean(final_sentence_table.column('Percentage')))
print('Standard Deviation:')
print(np.std(final_sentence_table.column('Percentage')))

array_percent_sent = final_sentence_table.column('Percentage') 

standard_units(array_percent_sent)

sentiment_su_sentence_table = final_sentence_table.with_column(
                            'SU', standard_units(array_percent_sent)).select(['Sentence Number', 'Total', 'Percentage', 'SU'])
sentiment_su_sentence_table.sort('SU', descending=True)

In [ ]:
print('Average Percentage of Sentiment Words:') 
print(np.mean(final_chapter_table.column('Percentage')))
print('Standard Deviation:')
print(np.std(final_chapter_table.column('Percentage')))

standard_units(final_chapter_table.column('Percentage'))

sentiment_su_chapter_table = final_chapter_table.with_column(
                    'SU', standard_units(final_chapter_table.column('Percentage'))).select([
                    'Chapter', 'Total Count', 'Percentage', 'SU'])
sentiment_su_chapter_table.sort('SU', descending=True)

## Standard Units for Concrete Words

In [ ]:
print('Average Percentage of Concrete Words:') 
print(np.mean(concreteness_table_sentence.column('Percentage')))
print('Standard Deviation:')
print(np.std(concreteness_table_sentence.column('Percentage')))

standard_units(concreteness_table_sentence.column('Percentage'))

concrete_su_sentence_table = concreteness_table_sentence.with_column(
                            'SU',standard_units(concreteness_table_sentence.column('Percentage'))).select([
                            'Sentence Number', 'Total Count', 'Percentage', 'SU'])
concrete_su_sentence_table.sort('SU', descending=True)

In [ ]:
print('Average Percentage of Concrete Words:') 
print(np.mean(concreteness_table_chapter.column('Percentage')))
print('Standard Deviation:')
print(np.std(concreteness_table_chapter.column('Percentage')))

concrete_su_chapter_table = concreteness_table_chapter.with_column(
                            'SU', standard_units(concreteness_table_chapter.column('Percentage'))).select([
                            'Chapter', 'Total Count', 'Percentage', 'SU'])
concrete_su_chapter_table.sort('SU', descending=True)

## Data

In [ ]:
#The following cell is my data for sentiment in sentences. 

In [ ]:
sentiment_su_sentence_table.where(sentiment_su_sentence_table.column('SU') >= 4).sort('SU', descending=True)

In [ ]:
final_sentence_table.where(final_sentence_table.column('Percentage') > .35).sort('Percentage', descending=True)

In [ ]:
sentence_number = [i for i in range(len(sentence_list))]
sentiment_su_sentence = sentiment_su_sentence_table.column('SU')
plt.plot(sentence_number, sentiment_su_sentence)
plt.ylabel('Sentiment in SU')
plt.xlabel('Sentence in Text')

In [ ]:
#The following cell is my data for sentiment in chapters. 

In [ ]:
sentiment_su_chapter_table.where(sentiment_su_chapter_table.column('SU') >= 1).sort('SU', descending=True)

In [ ]:
sentiment_su_chapter_table.where(sentiment_su_chapter_table.column('SU') <= -1).sort('SU')

In [ ]:
reduced_sent = sentiment_su_chapter_table.with_column('Chapter Number', np.arange(1, 38, 1)).select(['Chapter Number', 'SU'])

In [ ]:
CHAPTERS_SENT = reduced_sent.column('Chapter Number')
SU_SENT = reduced_sent.column('SU')
plt.plot(CHAPTERS_SENT, SU_SENT)
plt.ylabel('Sentiment in SU')
plt.xlabel('Chapter in Text')

In [ ]:
#The following cell is my data for concreteness in sentences.

In [ ]:
concrete_su_sentence_table.where(concrete_su_sentence_table.column('SU') >= 5).sort('SU', descending=True).show()

In [ ]:
concreteness_table_sentence.where(concreteness_table_sentence.column('Percentage') > .15)

In [ ]:
concrete_su_sentence = concrete_su_sentence_table.column('SU')
plt.plot(sentence_number, concrete_su_sentence)
plt.ylabel('Concreteness in SU')
plt.xlabel('Sentence in Text')

In [ ]:
#The following cell is my data for concreteness in chapters.

In [ ]:
print(np.max(concrete_su_chapter_table.column('SU')))
concrete_su_chapter_table.where(concrete_su_chapter_table.column('SU') >= 1).sort('SU', descending=True)

In [ ]:
print(np.min(concrete_su_chapter_table.column('SU')))
concrete_su_chapter_table.where(concrete_su_chapter_table.column('SU') <= -1).sort('SU')

In [ ]:
reduced = concrete_su_chapter_table.with_column('Chapter Number', np.arange(1, 38, 1)).select(['Chapter Number', 'SU'])
CHAPTERS_CONCRETE = reduced.column('Chapter Number')
SU_CONCRETE = reduced.column('SU')
plt.plot(CHAPTERS_CONCRETE, SU_CONCRETE)
plt.ylabel('Concrete in SU')
plt.xlabel('Chapter in Text')

In [ ]:
plt.plot(CHAPTERS_SENT, SU_SENT)
plt.plot(CHAPTERS_CONCRETE, SU_CONCRETE)
plt.ylabel('Sentiment or Concrete in Text')
plt.xlabel('Chapter in Text')

## Looking More In-Depth at Chapters 32 and 33

In [ ]:
chapter_1_segment = final_chapter_table.where(final_chapter_table.column('Chapter') == 'Chapter 1').column('Segment')
chapter_1_sentences = sent_tokenize(str(chapter_1_segment))
len(chapter_1_sentences)

In [ ]:
def length_chapter(label):
    chapter_segment = final_chapter_table.where(final_chapter_table.column('Chapter') == label).column('Segment')
    chapter_sentences = sent_tokenize(str(chapter_segment))
    return len(chapter_sentences)

sentences_in_chapter = final_chapter_table.apply(length_chapter, 'Chapter')
final_chapter_table.append_column('Sentences in Chapter', sentences_in_chapter)

final_chapter_table.where(final_chapter_table.column('Chapter') == 'Chapter 32')
print(sentences_in_chapter)
print(sum(sentences_in_chapter) - (146 + 93 + 178 + 282 + 158 + 16))

final_sentence_table.where(np.logical_and(final_sentence_table.column('Sentence Number') >= 6620, 
                                         final_sentence_table.column('Sentence Number') <= 6628))
final_sentence_table.where(final_sentence_table.column('Sentence Number') == 6599)

In [ ]:
chapter_32_sentiment = sentiment_su_sentence_table.where(np.logical_and(sentiment_su_sentence_table.column('Sentence Number') >= 6597, 
                                                sentiment_su_sentence_table.column('Sentence Number') <= 6613)).sort('SU', descending=True)
chapter_32_sentiment

In [ ]:
chapter_32_concrete = concrete_su_sentence_table.where(np.logical_and(concrete_su_sentence_table.column('Sentence Number') >= 6597, 
                                               concrete_su_sentence_table.column('Sentence Number') <= 6613)).sort('SU', descending=True)
chapter_32_concrete

In [ ]:
chapter_33_sentiment = sentiment_su_sentence_table.where(np.logical_and(sentiment_su_sentence_table.column('Sentence Number') > 6613, 
                                                sentiment_su_sentence_table.column('Sentence Number') <= 6759)).sort('SU')
chapter_33_sentiment

In [ ]:
chapter_33_concrete = concrete_su_sentence_table.where(np.logical_and(concrete_su_sentence_table.column('Sentence Number') > 6620, 
                                                                     concrete_su_sentence_table.column('Sentence Number') <= 6625))
chapter_33_concrete.sort('SU', descending=True)

## Extra: Visualizing the Chapter Data (Before SU)

In [ ]:
#Example graph from the internet: 
import matplotlib.pyplot as plt
plt.plot([1,2,3,4])
plt.ylabel('some numbers')
example = plt.savefig('example.png')

In [ ]:
chapter_number = [i+1 for i in range(len(passage_array))]
len(chapter_number)

In [ ]:
chapter_number = [i+1 for i in range(len(chapter_rows))]
plt.plot(chapter_number, chapter_percentage)
plt.ylabel('Percentage of Sentiment')
plt.xlabel('Chapter in Novel')

In [ ]:
short_table = final_chapter_table.with_column('Chapter Number', chapter_number).select(['Chapter Number', 'Percentage'])
short_table.scatter('Chapter Number')

In [ ]:
array = np.array(chapter_number)
chapters_part_one = array.take(np.arange(0, 11))
percents_part_one = chapter_percentage.take(np.arange(0, 11))
plt.plot(chapters_part_one, percents_part_one)
plt.ylabel('Percentage of Sentiment')
plt.xlabel('Chapter in Part One')

In [ ]:
chapters_part_two = array.take(np.arange(12, 32))
percents_part_two = chapter_percentage.take(np.arange(12, 32))
plt.plot(chapters_part_two, percents_part_two)
plt.ylabel('Percentage of Sentiment')
plt.xlabel('Chapter in Part Two')

In [ ]:
chapters_part_three = array.take(np.arange(33, 37))
percents_part_three = chapter_percentage.take(np.arange(33, 37))
plt.plot(chapters_part_three, percents_part_three)
plt.ylabel('Percentage of Sentiment')
plt.xlabel('Chapter in Part Three')

## Extra: Visualizing the Sentence Data (Before SU)

In [ ]:
sentence_number = [i+1 for i in range(len(sentence_list))]

In [ ]:
plt.plot(sentence_number, sentence_percentage)
plt.ylabel('Percentage of Sentiment Words')
plt.xlabel('Sentence in Text')

In [ ]:
array = np.array(sentence_number)
sentence_array = array.take(np.arange(1200, 1400))
percentage_array = percentage.take(np.arange(1200, 1400))
plt.plot(sentence_array, percentage_array)

In [ ]:
plt.plot(sentence_number, negative_sentence_words_len)

In [ ]:
plt.plot(sentence_number, positive_sentence_words_len)

In [ ]:
negative_percent = np.array(negative_sentence_words_len) / sentence_length
negative_percent
plt.plot(sentence_number, negative_percent)

In [ ]:
positive_percent = np.array(positive_sentence_words_len) / sentence_length
plt.plot(sentence_number, positive_percent)

In [ ]:
plt.plot(sentence_number, negative_percent)
plt.plot(sentence_number, positive_percent)

In [ ]:
final_sentence_table.with_column('Sentence Number', sentence_number).select(['Sentence Number', 'Percentage']).scatter('Sentence Number')

## Visualizing Concrete Data

In [ ]:
plt.plot(sentence_number, concrete_percentage_sentence)
plt.ylabel('Percentage of Concrete Words')
plt.xlabel('Sentence in Text')

In [ ]:
plt.plot(chapter_number, concrete_percentage_chapter)
plt.ylabel('Percentage of Concrete Words')
plt.xlabel('Chapter in Text')

In [ ]:
concrete_part_one = concrete_percentage_chapter.take(np.arange(0, 11))
plt.plot(chapters_part_one, concrete_part_one)
plt.ylabel('Percentage of Concrete Words')
plt.xlabel('Chapter in Part One')

In [ ]:
concrete_part_two = concrete_percentage_chapter.take(np.arange(12, 32))
plt.plot(chapters_part_two, concrete_part_two)
plt.ylabel('Percentage of Concrete Words')
plt.xlabel('Chapter in Part Two')

In [ ]:
concrete_part_three = concrete_percentage_chapter.take(np.arange(33, 37))
plt.plot(chapters_part_three, concrete_part_three)
plt.ylabel('Percentage of Concrete Words')
plt.xlabel('Chapter in Part Three')

In [ ]:
concreteness_table_sentence.with_column('Sentence Number', sentence_number).select(['Sentence Number', 'Percentage']).scatter('Sentence Number')

In [ ]:
plt.plot(chapter_number, concrete_percentage_chapter)
plt.plot(chapter_number, chapter_percentage)